In [1]:
import numpy as np
import math
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import os
import torch
import torch.utils.data as data 

from omegaconf import OmegaConf
from torchvision import transforms
from torch.nn.parallel import DistributedDataParallel as DDP
# 
from contrastive_learning.tests.test_model import load_lin_model, predict_traj_actions
from contrastive_learning.tests.animate_markers import AnimateMarkers
from contrastive_learning.tests.animate_rvec_tvec import AnimateRvecTvec
from contrastive_learning.datasets.dataloaders import get_dataloaders

from contrastive_learning.models.custom_models import LinearInverse
from contrastive_learning.datasets.state_dataset import StateDataset
from contrastive_learning.tests.plotting import plot_rvec_tvec, plot_corners

### Model Loading
Create the distributed group
Load the linear inverse model from the saved path

In [2]:
# Start the multiprocessing to load the saved models properly
os.environ["MASTER_ADDR"] = "localhost"
os.environ["MASTER_PORT"] = "29504"

torch.distributed.init_process_group(backend='gloo', rank=0, world_size=1)
torch.cuda.set_device(0)

In [3]:
# Set the device and out_dir
device = torch.device('cuda:0')
out_dir = '/home/irmak/Workspace/DAWGE/contrastive_learning/out/2022.07.28/10-10_pli_ue_False_lf_mse_fi_1_pt_corners_bs_64_hd_64_lr_0.001_zd_8'
cfg = OmegaConf.load(os.path.join(out_dir, '.hydra/config.yaml'))
model_path = os.path.join(out_dir, 'models/lin_model.pt')

# Load the encoder
lin_model = load_lin_model(cfg, device, model_path)

In [4]:
print(lin_model)

DistributedDataParallel(
  (module): LinearInverse(
    (model): Sequential(
      (0): Linear(in_features=32, out_features=64, bias=True)
      (1): ReLU()
      (2): Linear(in_features=64, out_features=16, bias=True)
      (3): ReLU()
      (4): Linear(in_features=16, out_features=2, bias=True)
    )
  )
)


In [5]:
print(cfg)

{'agent': {'_target_': 'contrastive_learning.models.agents.pli.PLI', 'loss_fn': 'mse', 'use_encoder': False, 'model': '???', 'optimizer': '???'}, 'optimizer': {'_target_': 'torch.optim.Adam', 'params': '???', 'lr': '???', 'weight_decay': '???'}, 'model': {'_target_': 'contrastive_learning.models.custom_models.LinearInverse', 'input_dim': '???', 'action_dim': '???', 'hidden_dim': '???'}, 'pos_encoder': {'_target_': 'contrastive_learning.models.custom_models.PosToEmbedding', 'input_dim': '???', 'hidden_dim': '???', 'out_dim': '???'}, 'seed': 42, 'device': 'cuda', 'agent_type': 'pli', 'dataset_type': 'state', 'pos_type': 'corners', 'train_epochs': 1000, 'save_frequency': 10, 'train_dset_split': 0.8, 'batch_size': 64, 'lr': 0.001, 'weight_decay': 1e-05, 'z_dim': 8, 'pos_dim': 8, 'hidden_dim': 64, 'action_dim': 2, 'distributed': True, 'num_workers': 4, 'world_size': 1, 'num_gpus': 4, 'fps': 15, 'frame_interval': 1, 'video_type': 'color', 'experiment': '${agent_type}_ue_${agent.use_encoder}_

### Action Animation
1. Dump every predicted action for given data directory
2. Save the predicted and current action in a video

In [14]:
demo_name = 'box_marker_move_15'
exp_name = '{}_{}'.format(out_dir.split('/')[-2], out_dir.split('/')[-1])
data_dir = '/home/irmak/Workspace/DAWGE/src/dawge_planner/data/move_demos/{}'.format(demo_name)
dump_dir = '/home/irmak/Workspace/DAWGE/contrastive_learning/tests/animations'
dump_file = '{}_{}.mp4'.format(demo_name, exp_name)

fps = 15

In [15]:
print('dump_file: {}'.format(dump_file))

dump_file: box_marker_move_15_2022.07.28_10-10_pli_ue_False_lf_mse_fi_1_pt_corners_bs_64_hd_64_lr_0.001_zd_8.mp4


In [16]:
# Get the dataset
dataset = StateDataset(cfg, single_dir=True, single_dir_root=data_dir)
predicted_actions = np.zeros((len(dataset), 2))
test_loader = data.DataLoader(dataset, batch_size=cfg.batch_size, shuffle=False, num_workers=4)

for i, batch in enumerate(test_loader):
    curr_pos, next_pos, action = [b.to(device) for b in batch]
    pred_action = lin_model(curr_pos, next_pos)

    print('Actual Action \t Predicted Action')
    for j in range(len(action)):
        print('{}, \t{}'.format(np.around(dataset.denormalize_action(action[j][0].cpu().detach().numpy()), 2),
                                          dataset.denormalize_action(pred_action[j][0].cpu().detach().numpy())))
        predicted_actions[i*cfg.batch_size+j,:] = dataset.denormalize_action(pred_action[j][0].cpu().detach().numpy())

with open(os.path.join(data_dir, 'predicted_actions.npy'), 'wb') as f:
    np.save(f, predicted_actions)

Actual Action 	 Predicted Action
[ 0.  -0.3], 	[0.09926589 0.09706353]
[ 0.  -0.3], 	[0.09960191 0.09840764]
[ 0.  -0.3], 	[0.10094558 0.10378232]
[ 0.  -0.3], 	[0.10004813 0.10019249]
[ 0.  -0.3], 	[0.09722285 0.08889138]
[ 0.  -0.3], 	[0.09646831 0.08587321]
[0.15 0.3 ], 	[0.10058965 0.10235861]
[0.15 0.3 ], 	[0.10435215 0.11740859]
[0.15 0.3 ], 	[0.10636669 0.12546673]
[0.15 0.3 ], 	[0.10709914 0.12839656]
[0.15 0.3 ], 	[0.1096012  0.13840478]
[0.15 0.3 ], 	[0.11459812 0.15839246]
[0.15 0.3 ], 	[0.11863635 0.1745454 ]
[ 0.  -0.3], 	[0.11931906 0.17727621]
[ 0.  -0.3], 	[0.11941175 0.177647  ]
[ 0.  -0.3], 	[0.12009514 0.18038057]
[0.15 0.3 ], 	[0.1230958 0.1923832]
[0.15 0.3 ], 	[0.12269313 0.19077252]
[0.15 0.3 ], 	[0.1242359 0.1969436]
[0.15 0.3 ], 	[0.11456321 0.15825284]
[0.15 0.3 ], 	[0.10570912 0.12283645]
[0.15 0.3 ], 	[0.09817298 0.09269193]
[0.15 0.3 ], 	[0.09003716 0.06014865]
[0.15 0.3 ], 	[0.08321918 0.0328767 ]
[ 0.  -0.3], 	[0.07707541 0.00830162]
[ 0.  -0.3], 	[ 0.073

In [17]:

if cfg.pos_type == 'corners':
    AnimateMarkers(
        data_dir = data_dir, 
        dump_dir = dump_dir, 
        dump_file = dump_file, 
        fps = fps,
        mult_traj = False,
        show_predicted_action = True 
    ) # Saves them in the given dump_file
elif cfg.pos_type == 'rvec_tvec':
    AnimateRvecTvec(
        data_dir = data_dir, 
        dump_dir = dump_dir, 
        dump_file = dump_file,
        fps = fps,
        show_predicted_action=True
    )

Starting Animation


100%|█████████████████████████████████████████████████████████████████████████████████| 63/63 [00:09<00:00,  6.37it/s]

Animation saved to: /home/irmak/Workspace/DAWGE/contrastive_learning/tests/animations/box_marker_move_15_2022.07.28_10-10_pli_ue_False_lf_mse_fi_1_pt_corners_bs_64_hd_64_lr_0.001_zd_8.mp4


### Action Prediction Image
Predict the action for each frame in the test dataset and dump them in a grid image

In [10]:
cfg.batch_size = 32
train_loader, test_loader, dataset = get_dataloaders(cfg)

In [11]:
len(test_loader.dataset)
batch = next(iter(test_loader))
curr_pos, next_pos, action = [b.to(device) for b in batch]
pred_action = lin_model(curr_pos, next_pos)

curr_pos.shape

torch.Size([32, 16])

In [12]:
print(curr_pos[0])

tensor([0.3576, 0.2308, 0.3382, 0.2531, 0.3155, 0.2350, 0.3366, 0.2140, 0.2905,
        0.2713, 0.3091, 0.2951, 0.2832, 0.3133, 0.2646, 0.2881],
       device='cuda:0')


In [13]:
nrows = 4
ncols = int(cfg.batch_size / nrows)
fig, axs = plt.subplots(figsize=(ncols*10,nrows*10), nrows=nrows, ncols=ncols) # Draw the predicted action
print(axs.shape)

for i in range(cfg.batch_size):
    axs_row = int(i / nrows)
    axs_col = int(i % nrows)
    
    action_np = dataset.denormalize_action(action[i].cpu().detach().numpy())
    pred_action_np = dataset.denormalize_action(pred_action[i].cpu().detach().numpy())
    
    if cfg.pos_type == 'corners':
        curr_pos_np = dataset.denormalize_corner(curr_pos[i].cpu().detach().numpy())
        plot_corners(
            ax=axs[axs_col, axs_row],
            curr_pos=curr_pos_np,
            use_img=False,
            img=None,
            plot_action=True,
            actions=(action_np, pred_action_np))
    elif cfg.pos_type == 'rvec_tvec':
        curr_pos_np = dataset.denormalize_pos_rvec_tvec(curr_pos[i].cpu().detach().numpy())
        plot_rvec_tvec(
            ax=axs[axs_col, axs_row],
            curr_pos=curr_pos_np,
            use_img=False,
            img=None,
            plot_action=True,
            actions=(action_np, pred_action_np))
        
plt.savefig(os.path.join(out_dir, 'pil_action_test.jpg'))
    

(4, 8)
